### **Create Virtual Environment and Install Packages**

In [5]:
!py -m venv .test_env

In [ ]:
%pip install requirements.txt

### **Download YOLOv7 Pose**

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt

### **Clone Repo For YOLOv7**

In [17]:
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...


### **Move this Notebook to yolov7 directory (Cloned above)**

### **Creating Results Directory**

In [3]:
import os
os.makedirs('../results', exist_ok=True)

### **Algorithm for Human Fall Detection**

In [4]:
import cv2
import math
import torch
from torchvision import transforms
import numpy as np
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weights = torch.load('../yolov7-w6-pose.pt', map_location=device)
model = weights['model']
_ = model.float().eval()
if torch.cuda.is_available():
    model = model.half().to(device)
# model = model.to(device)

_ = model.eval()

# Open a connection to the webcam (usually 0 for the default webcam)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Can't open webcam.")
    exit()

result_img_num = 1
# Loop to continuously capture frames from the webcam
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Convert frame to RGB
    orig_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_width = int(cap.get(3))
    image = letterbox(orig_image, (frame_width), stride=64, auto=True)[0]

    # Convert image data to device
    image = transforms.ToTensor()(image)
    image = torch.tensor(np.array([image.numpy()]))
    if torch.cuda.is_available():
        image = image.half().to(device)
    # image = image.to(device)

    # Get predictions
    with torch.no_grad():
        output, _ = model(image)

    # Apply non-maximum suppression
    output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
    output = output_to_keypoint(output)
    im0 = image[0].permute(1, 2, 0) * 255
    im0 = im0.cpu().numpy().astype(np.uint8)

    # Reshape image format to BGR
    im0 = cv2.cvtColor(im0, cv2.COLOR_RGB2BGR)
    for idx in range(output.shape[0]):
        xmin, ymin = (output[idx, 2]-output[idx, 4]/2), (output[idx, 3]-output[idx, 5]/2)
        xmax, ymax = (output[idx, 2]+output[idx, 4]/2), (output[idx, 3]+output[idx, 5]/2)

        left_shoulder_y = output[idx][23]
        left_shoulder_x = output[idx][22]
        right_shoulder_y = output[idx][26]

        left_body_y = output[idx][41]
        left_body_x = output[idx][40]
        right_body_y = output[idx][44]

        len_factor = math.sqrt(((left_shoulder_y - left_body_y)**2 + (left_shoulder_x - left_body_x)**2))

        left_foot_y = output[idx][53]
        right_foot_y = output[idx][56]

        if left_shoulder_y > left_foot_y - len_factor and left_body_y > left_foot_y - (len_factor / 2) and left_shoulder_y > left_body_y - (len_factor / 2):
            cv2.rectangle(im0, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color=(0, 0, 255),
                          thickness=5, lineType=cv2.LINE_AA)
            cv2.putText(im0, 'Human Fell', (11, 100), 0, 1, [0, 0, 2550], thickness=3, lineType=cv2.LINE_AA)
            result_filename = f"../results/human_fell_{result_img_num}.jpg"
            cv2.imwrite(result_filename, im0)
            result_img_num += 1

    cv2.imshow('WebCam', im0)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
